In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate
!pip install transformers
import accelerate
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/data

In [36]:
# label_df = result_df[['Prompt','Protagonist','Antagonist','Innocent']]
import pandas as pd
import string
import re

label_df = pd.read_csv("/content/mainnnnanananananana.csv")
# label_df = (
#     label_df.groupby('fine_grained_role1')
#     .head(90)
#     .reset_index(drop=True)
# )
def limit_specific_labels(df, labels_to_limit, max_count=90):
    # Iterate through the specified labels to limit
    for label in labels_to_limit:
        if label in df.columns:
            # Find rows where the label is 1
            ones = df[df[label] == 1]

            # If there are more than 'max_count' occurrences, downsample
            if len(ones) > max_count:
                sampled_ones = ones.sample(n=max_count, random_state=42)  # Randomly sample 90 rows
                other_rows = df[df[label] == 0]  # Keep rows where the label is 0
                df = pd.concat([sampled_ones, other_rows], axis=0)

    # Shuffle the resulting dataframe (optional, for randomness)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    return df

# List of labels to limit the "1"s to 90 occurrences
labels_to_limit = ["Instigator", "Foreign Adversary", "Deceiver"]

# Apply the function to your dataframe
df_balanced = limit_specific_labels(label_df, labels_to_limit)

# Display the balanced dataframe
label_df = df_balanced.copy()

label_df = label_df.sample(frac=1, random_state=42).reset_index(drop=True)

def clean_text(text):
    text = ' '.join(text.split())
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

label_df['Prompt'] = label_df['Prompt'].apply(clean_text)
label_df

Prompt  Bigot  Conspirator  \
0     The deceitful Vladimir Putin a ruthless antago...      0            0   
1     Johnson championing a swift resolution for the...      0            0   
2     Hillary Clinton seemingly an innocent bystande...      0            0   
3     Though the loss of influence over Germany is u...      0            0   
4     n who controls the world’s largest nuclear ars...      0            0   
...                                                 ...    ...          ...   
1921  As a champion for global food security the Wor...      0            0   
1922  In the English language the proRussia perspect...      0            0   
1923  While Lula champions dedollarization mirroring...      0            0   
1924  Sir Richard Kemp a staunch supporter of Ukrain...      0            0   
1925  Al Gore a selfproclaimed environmental savior ...      0            0   

      Corrupt  Deceiver  Exploited  Foreign Adversary  Forgotten  Guardian  \
0           0         1          0                  0          0         0   
1           0         0          0                  0          0         0   
2           0         0          0                  0          0         0   
3           0         0          0                  0          0         0   
4           1         0          0                  0          0         0   
...       ...       ...        ...                ...        ...       ...   
1921        0         0          0                  0          0         0   
1922        0         0          0                  0          0         0   
1923        0         0          0                  0          0         0   
1924        0         0          0                  0          0         0   
1925        0         1          0                  0          0         0   

      Incompetent  ...  Rebel  Saboteur  Scapegoat  Spy  Terrorist  Traitor  \
0               0  ...      0         0          0    0          0        0   
1               0  ...      0         0          0    0          0        0   
2               0  ...      0         0          1    0          0        0   
3               0  ...      0         0          1    0          0        0   
4               0  ...      0         0          0    0          0        0   
...           ...  ...    ...       ...        ...  ...        ...      ...   
1921            0  ...      0         0          0    0          0        0   
1922            0  ...      0         0          0    0          0        0   
1923            0  ...      1         0          0    0          0        0   
1924            0  ...      0         0          0    0          0        0   
1925            0  ...      0         0          0    0          0        0   

      Tyrant  Underdog  Victim  Virtuous  
0          0         0       0         0  
1          0         1       0         0  
2          0         0       0         0  
3          0         0       0         0  
4          0         0       0         0  
...      ...       ...     ...       ...  
1921       0         0       0         0  
1922       1         0       0         0  
1923       0         0       0         0  
1924       0         1       0         0  
1925       0         0       0         0  

[1926 rows x 23 columns]

In [13]:
train_df = label_df[:1500]
validation_df = label_df[1500:]
test_df = label_df[950:]
train_labels_df = label_df[:1500]
validation_labels_df = label_df[1500:]
test_labels_df = label_df[950:]


In [14]:
train_sentences = train_df['Prompt'].values.tolist()
train_labels = train_labels_df.iloc[:, 1:].values
train_labels = train_labels.astype(np.float32)

validation_sentences = validation_df['Prompt'].values.tolist()
validation_labels = validation_labels_df.iloc[:, 1:].values
validation_labels = validation_labels.astype(np.float32)

In [15]:
num_columns = train_labels.shape[1]
print("Number of columns:", num_columns)

Number of columns: 22


In [16]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
# model

from transformers import BartForSequenceClassification, AutoTokenizer

checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BartForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [17]:
class ArgDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [18]:
train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
train_dataset = ArgDataset(train_encodings, train_labels)

validation_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
validation_dataset = ArgDataset(validation_encodings, validation_labels)

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions[0] > 0.5).astype(int)

    # Compute exact match ratio
    # exact_match_ratio = (preds == labels).all(axis=1).mean()

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    # Generate classification report
    # report = classification_report(labels, preds, target_names=label_names, zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'classification_report': report
    }


In [21]:
# import os
# os.environ["WANDB_DISABLED"] = "true"

In [23]:
from transformers import Trainer, TrainingArguments,DefaultFlowCallback
import wandb
wandb.init(mode="disabled")

training_args = TrainingArguments(
    output_dir= './results',
    eval_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs = 5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    logging_steps=50,
    metric_for_best_model="accuracy",
    remove_unused_columns=False,
    report_to = [],
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[DefaultFlowCallback]
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sa

TrainOutput(global_step=940, training_loss=0.05796046745269857, metrics={'train_runtime': 2243.7948, 'train_samples_per_second': 3.343, 'train_steps_per_second': 0.419, 'total_flos': 4808019662820000.0, 'train_loss': 0.05796046745269857, 'epoch': 5.0})

In [ ]:
import matplotlib.pyplot as plt

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Initialize lists for storing values
losses = []
accuracies = []
precisions = []
f1_scores = []
recalls = []

# Parse through the log history and extract values
for entry in log_history:
    if 'eval_loss' in entry and 'eval_accuracy' in entry and 'eval_precision' in entry and 'eval_recall' in entry and 'eval_f1' in entry:
        losses.append(entry['eval_loss'])
        accuracies.append(entry['eval_accuracy'])
        precisions.append(entry['eval_precision'])
        f1_scores.append(entry['eval_f1'])
        recalls.append(entry['eval_recall'])

# Define epochs (adjust based on your actual epoch range)
epochs = list(range(1, len(losses) + 1))

# Create a figure and axis for plotting
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot the first metric (loss) on the primary y-axis
ax1.plot(epochs, losses, color='tab:orange', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

# Create a secondary y-axis for Accuracy
ax2 = ax1.twinx()
ax2.plot(epochs, accuracies, color='tab:blue', linestyle='--', label='Accuracy')
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Create a tertiary y-axis for Precision
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third y-axis
ax3.plot(epochs, precisions, color='tab:green', linestyle='-', label='Precision')
ax3.set_ylabel('Precision', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

# Create a fourth y-axis for Recall
ax4 = ax1.twinx()
ax4.spines['right'].set_position(('outward', 120))  # Offset the fourth y-axis
ax4.plot(epochs, recalls, color='tab:red', linestyle=':', label='Recall')
ax4.set_ylabel('Recall', color='tab:red')
ax4.tick_params(axis='y', labelcolor='tab:red')

# Create a fifth y-axis for F1-score
ax5 = ax1.twinx()
ax5.spines['right'].set_position(('outward', 180))  # Offset the fifth y-axis
ax5.plot(epochs, f1_scores, color='tab:purple', linestyle='-.', label='F1-Score')
ax5.set_ylabel('F1-Score', color='tab:purple')
ax5.tick_params(axis='y', labelcolor='tab:purple')

# Add legends for each line
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
lines4, labels4 = ax4.get_legend_handles_labels()
lines5, labels5 = ax5.get_legend_handles_labels()

ax2.legend(lines1 + lines2 + lines3 + lines4 + lines5,
           labels1 + labels2 + labels3 + labels4 + labels5,
           loc='lower left', bbox_to_anchor=(1.05, 1))

# Title and grid
plt.title('Validation Metrics vs Epochs')
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [24]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")
# model = BertForSequenceClassification.from_pretrained("/content/")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(


('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.json',
 '/content/merges.txt',
 '/content/added_tokens.json',
 '/content/tokenizer.json')

In [25]:
model = BartForSequenceClassification.from_pretrained("/content/", num_labels=22, problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("/content/")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11', '12': 'LABEL_12', '13': 'LABEL_13', '14': 'LABEL_14', '15': 'LABEL_15', '16': 'LABEL_16', '17': 'LABEL_17', '18': 'LABEL_18', '19': 'LABEL_19', '20': 'LABEL_20', '21': 'LABEL_21'}. The number of labels wil be overwritten to 22.


In [ ]:
# test_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
test_df

In [26]:
import pandas as pd
import requests
import numpy as np

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-entity-mentions.txt"

# Load the annotations file
df = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'entity_mention', 'start_offset', 'end_offset'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = base_url + article_id  # Construct the file URL
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.text  # Return the content if the file is found
    else:
        return f"File {article_id} not found"

df['article_content'] = df['article_id'].apply(fetch_file_content)

def extract_contextual_content(row):
    content = row['article_content']
    start = row['start_offset']
    end = row['end_offset']

    if isinstance(content, str) and len(content) >= end:
        # Determine the range for 'before' based on the start position
        begin = max(0,start-250)
        finish = max(len(content),end+250)
        before = content[begin:start]
        after = content[end:finish]
        sentence = before + row['entity_mention'] + after

        return sentence


df['contextual_content'] = df.apply(extract_contextual_content, axis=1)
dedf = df.copy()
dedf.insert(0, 'Index', range(1, len(dedf) + 1))
dedf

Index            article_id        entity_mention  start_offset  \
0       1  EN_UA_DEV_100012.txt            Washington          1441   
1       2  EN_UA_DEV_100012.txt             Americans          2397   
2       3      EN_CC_200053.txt           Rishi Sunak            85   
3       4      EN_CC_200040.txt  Extinction Rebellion           189   
4       5      EN_CC_200070.txt         Eric Garcetti            89   
..    ...                   ...                   ...           ...   
86     87  EN_UA_DEV_100002.txt              Zelensky           657   
87     88  EN_UA_DEV_100002.txt               Ukraine           672   
88     89  EN_UA_DEV_100002.txt                  West           725   
89     90  EN_UA_DEV_100002.txt                  NATO          1027   
90     91  EN_UA_DEV_100002.txt                Russia          1200   

    end_offset                                    article_content  \
0         1450  Ukraine's Minerals: What the West is Fighting ...   
1         2405  Ukraine's Minerals: What the West is Fighting ...   
2           95  UK’s Sunak Reverses Decision to Skip COP27 Cli...   
3          208  Climate Protesters Out Of Control As They Atta...   
4          101  Wat? L.A. Mayor Garcetti Flies to Argentina to...   
..         ...                                                ...   
86         664  Ukrainian nationalism, Ukrainian patriotism wi...   
87         678  Ukrainian nationalism, Ukrainian patriotism wi...   
88         728  Ukrainian nationalism, Ukrainian patriotism wi...   
89        1030  Ukrainian nationalism, Ukrainian patriotism wi...   
90        1205  Ukrainian nationalism, Ukrainian patriotism wi...   

                                   contextual_content  
0   s\n\nUkraine's Minerals: What the West is Figh...  
1   ources, said Vladimir Dzhabarov, official of t...  
2   UK’s Sunak Reverses Decision to Skip COP27 Cli...  
3   Climate Protesters Out Of Control As They Atta...  
4   Wat? L.A. Mayor Garcetti Flies to Argentina to...  
..                                                ...  
86  raine is winning the war and propaganda about ...  
87  g the war and propaganda about how could y\n\n...  
88  with the entire NATO supporting them… fact is ...  
89  ng it.\n\nThe only country that doesn’t want U...  
90  ction, simply devastation. Then what? What is ...  

[91 rows x 7 columns]

In [27]:
import torch
import pandas as pd
from transformers import BartForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
# model = BartForSequenceClassification.from_pretrained("/content/", num_labels=22, problem_type="multi_label_classification")
# tokenizer = AutoTokenizer.from_pretrained("/content/")

# List to store predictions
predictions = []

# Assuming dedf contains the data you want to classify
for idx, row in dedf.iterrows():
    arg_id = row['Index']
    text = row['contextual_content']

    # Tokenizing the input text
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Disable gradient computation during inference
    with torch.no_grad():
        outputs = model(**encoding)

    # Apply Sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    probabilities = sigmoid(outputs.logits).flatten()

    # Get the index of the label with the maximum probability
    max_label_index = torch.argmax(probabilities).item()

    # Append the result with the argument ID and maximum probability label
    predictions.append([arg_id, max_label_index])

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Index', 'Max_Prob_Label'])

# Display the predictions
print(predictions_df)


    Index  Max_Prob_Label
0       1               5
1       2               5
2       3               8
3       4              13
4       5               7
..    ...             ...
86     87               9
87     88               9
88     89               9
89     90               9
90     91               9

[91 rows x 2 columns]


In [28]:
indxxx = ['Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']

In [29]:
predictions_df['Predicted_Label'] = predictions_df['Max_Prob_Label'].map(lambda x: indxxx[x])
predictions_df = predictions_df.drop(columns='Max_Prob_Label')
predictions_df

Index    Predicted_Label
0       1  Foreign Adversary
1       2  Foreign Adversary
2       3        Incompetent
3       4           Saboteur
4       5           Guardian
..    ...                ...
86     87         Instigator
87     88         Instigator
88     89         Instigator
89     90         Instigator
90     91         Instigator

[91 rows x 2 columns]

In [30]:
# # predictions_df.to_csv('checker.csv', sep="\t", index=False, header=False)
# # predictions_df

# columns_to_check = predictions_df.columns[1:]
# has_single_one = (predictions_df[columns_to_check].sum(axis=1) > 0)

# df['has_single_one'] = has_single_one

# df[has_single_one].count()


In [ ]:
#predictions_df.valuecounts
#print value counts
df = predictions_df.copy()
df['article_id'] = dedf['article_id']
df = df.drop(columns=['Index'])
df['fine_grained'] = predictions_df['Predicted_Label']

# df.insert(0, 'article_id', dedf['article_id'])
df.insert(1, 'entity_mention', dedf['entity_mention'])
df.insert(2, 'start_offset', dedf['start_offset'])
df.insert(3, 'end_offset', dedf['end_offset'])
df.insert(4, 'label', 'Antagonist')


# Sample dataframe (assuming 'df' is your dataframe)
# Create a dictionary to map fine-grained roles to their labels
role_to_label = {
    'Guardian': 'Protagonist',
    'Martyr': 'Protagonist',
    'Peacemaker': 'Protagonist',
    'Rebel': 'Protagonist',
    'Underdog': 'Protagonist',
    'Virtuous': 'Protagonist',
    'Instigator': 'Antagonist',
    'Conspirator': 'Antagonist',
    'Tyrant': 'Antagonist',
    'Foreign Adversary': 'Antagonist',
    'Traitor': 'Antagonist',
    'Spy': 'Antagonist',
    'Saboteur': 'Antagonist',
    'Corrupt': 'Antagonist',
    'Incompetent': 'Antagonist',
    'Terrorist': 'Antagonist',
    'Deceiver': 'Antagonist',
    'Bigot': 'Antagonist',
    'Forgotten': 'Innocent',
    'Exploited': 'Innocent',
    'Victim': 'Innocent',
    'Scapegoat': 'Innocent'
}

# Function to map fine_grained role to the correct label
def assign_label(fine_grained_value):
    roles = fine_grained_value.split(';')  # Split multiple roles by semicolon
    # For each role, assign its corresponding label
    labels = [role_to_label.get(role.strip(), 'Unknown') for role in roles]
    return labels

df = df[['article_id', 'entity_mention', 'start_offset', 'end_offset','label', 'fine_grained']]

# 5. Save to a tab-separated text file
df.to_csv('outputers6.txt', sep='\t', index=False)

# Display the result
df


article_id        entity_mention  start_offset  end_offset  \
0   EN_UA_DEV_100012.txt            Washington          1441        1450   
1   EN_UA_DEV_100012.txt             Americans          2397        2405   
2       EN_CC_200053.txt           Rishi Sunak            85          95   
3       EN_CC_200040.txt  Extinction Rebellion           189         208   
4       EN_CC_200070.txt         Eric Garcetti            89         101   
..                   ...                   ...           ...         ...   
86  EN_UA_DEV_100002.txt              Zelensky           657         664   
87  EN_UA_DEV_100002.txt               Ukraine           672         678   
88  EN_UA_DEV_100002.txt                  West           725         728   
89  EN_UA_DEV_100002.txt                  NATO          1027        1030   
90  EN_UA_DEV_100002.txt                Russia          1200        1205   

         label       fine_grained  
0   Antagonist  Foreign Adversary  
1   Antagonist  Foreign Adversary  
2   Antagonist        Incompetent  
3   Antagonist           Saboteur  
4   Antagonist           Guardian  
..         ...                ...  
86  Antagonist         Instigator  
87  Antagonist         Instigator  
88  Antagonist         Instigator  
89  Antagonist         Instigator  
90  Antagonist         Instigator  

[91 rows x 6 columns]